In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import sys
sys.path.insert(0,'../tools')
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn import svm
import copy
from multiprocessing import Pool
from sklearn.metrics import roc_curve, auc
import time
import utils
import os
from joblib import dump, load

In [2]:
import utils
import imp

# Lemmatize the text

In [4]:
import spacy
nlp = spacy.load('en',disable = ["parser", "ner"])
def lemma(doc):
    doc = nlp(doc)
    lemmatized_doc = ''
    for token in doc:
        if not token.lemma_ == '-PRON-':
            lemmatized_doc += ' ' + token.lemma_
    return lemmatized_doc

In [5]:
from tqdm import tqdm
lemmatized_text = []
for doc in tqdm(new_text_total):
    doc = lemma(doc)
    lemmatized_text.append(doc)

100%|██████████| 5334/5334 [05:33<00:00, 15.98it/s]


In [6]:
new_text_train = lemmatized_text[:train_len]
label_train = label_total[:train_len]
new_text_test = lemmatized_text[train_len:]
label_test = label_total[train_len:]

In [7]:
print(len(label_train))

4267


In [8]:
des_dir = '../preprocessing/result/pu_ratio2/'
pickle.dump(new_text_train,open(des_dir + 'text_train_lemma.pkl','wb'))
pickle.dump(new_text_test,open(des_dir + 'text_test_lemma.pkl','wb'))

# When you already have lemmatized text

In [14]:
src_dir = '../preprocessing/result/pu_ratio2/'
new_text_train = pickle.load(open(src_dir + 'text_train_lemma.pkl','rb'))
new_text_test = pickle.load(open(src_dir + 'text_test_lemma.pkl','rb'))

In [15]:
new_text_train[0][500:700]

'w return with fever to 102 F. here in the ed be .4 F. Patient conversant , however not sure why be here . be unable to report any problem . during last hospital stay , Mr. be reintubate once for aspir'

In [16]:
new_text_total[0][500:700]

' returns with fevers to 102 F. Here in the ED he is .4 F. Patient conversant, however not sure why he is here. He is unable to report any problems. During his last hospital stay, Mr. was reintubated o'

# 1,2 gram SVM

In [17]:
Tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,2),min_df =5,stop_words='english',max_df = 0.8)
X_train = Tfidf_vectorizer.fit_transform(new_text_train)
Y_train = label_total[:train_len]
X_test = Tfidf_vectorizer.transform(new_text_test)
Y_test = label_total[train_len:]
print(np.shape(X_train))

(4267, 124451)


In [18]:
SVC_clf = svm.SVC(class_weight = 'balanced',kernel = 'linear',probability=1)
SVC_clf.fit(X_train, Y_train)
y_pred = SVC_clf.predict(X_test)
y_pred_proba = SVC_clf.predict_proba(X_test)
y_pred = np.argmax(y_pred_proba,axis=-1)
print(metrics.classification_report(Y_test, y_pred, target_names=['No_PU','PU']))
print(metrics.confusion_matrix(Y_test,y_pred))
print(metrics.roc_auc_score(Y_test, y_pred))

              precision    recall  f1-score   support

       No_PU       0.87      0.90      0.89       711
          PU       0.79      0.73      0.76       356

    accuracy                           0.84      1067
   macro avg       0.83      0.82      0.82      1067
weighted avg       0.84      0.84      0.84      1067

[[642  69]
 [ 97 259]]
0.8152408381927654


In [25]:
print(metrics.roc_auc_score(Y_test, y_pred_proba[:,1]))

0.9240032238183283


In [26]:
dump(SVC_clf, 'pu_clf_12gram.joblib')

['pu_clf_12gram.joblib']

In [ ]:
SVC_clf = svm.SVC(,probability=1)
SVC_clf.fit(X_train, Y_train)
y_pred = SVC_clf.predict(X_test)
y_pred_proba = SVC_clf.predict_proba(X_test)
y_pred = np.argmax(y_pred_proba,axis=-1)
print(metrics.classification_report(Y_test, y_pred, target_names=['No_PU','PU']))
print(metrics.confusion_matrix(Y_test,y_pred))
print(metrics.roc_auc_score(Y_test, y_pred))

In [6]:
Tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,2),min_df =5,stop_words='english',max_df = 0.8)
X_train = Tfidf_vectorizer.fit_transform(utils.back_to_doc(text_train)[:train_len])
Y_train = label_total[:train_len]
X_test = Tfidf_vectorizer.transform(utils.back_to_doc(text_test))
Y_test = label_total[train_len:]
print(np.shape(X_train))

(4267, 125847)


In [7]:
SVC_clf = svm.SVC(class_weight = 'balanced',kernel = 'linear',probability=1)
SVC_clf.fit(X_train, Y_train)
y_pred = SVC_clf.predict(X_test)
y_pred_proba = SVC_clf.predict_proba(X_test)
y_pred = np.argmax(y_pred_proba,axis=-1)
print(metrics.classification_report(Y_test, y_pred, target_names=['No_PU','PU']))
print(metrics.confusion_matrix(Y_test,y_pred))
print(metrics.roc_auc_score(Y_test, y_pred))

              precision    recall  f1-score   support

       No_PU       0.86      0.91      0.89       711
          PU       0.80      0.71      0.75       356

    accuracy                           0.84      1067
   macro avg       0.83      0.81      0.82      1067
weighted avg       0.84      0.84      0.84      1067

[[648  63]
 [103 253]]
0.811033281183331


In [6]:
Count_vectorizer = CountVectorizer(min_df =5,stop_words='english',max_df = 0.8)
X_train = Count_vectorizer.fit_transform(utils.back_to_doc(text_train)[:train_len])
Y_train = label_total[:train_len]
X_test = Count_vectorizer.transform(utils.back_to_doc(text_test))
Y_test = label_total[train_len:]
print(np.shape(X_train))

(4267, 16116)


In [7]:
SVC_clf = svm.SVC(class_weight = 'balanced',kernel = 'linear',probability=1)
SVC_clf.fit(X_train, Y_train)
y_pred = SVC_clf.predict(X_test)
y_pred_proba = SVC_clf.predict_proba(X_test)
y_pred = np.argmax(y_pred_proba,axis=-1)
print(metrics.classification_report(Y_test, y_pred, target_names=['No_PU','PU']))
print(metrics.confusion_matrix(Y_test,y_pred))
print(metrics.roc_auc_score(Y_test, y_pred))

              precision    recall  f1-score   support

       No_PU       0.82      0.91      0.87       711
          PU       0.78      0.61      0.69       356

    accuracy                           0.81      1067
   macro avg       0.80      0.76      0.78      1067
weighted avg       0.81      0.81      0.81      1067

[[650  61]
 [138 218]]
0.7632824475734445


In [ ]:
Tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,2),min_df =5,stop_words='english',max_df = 0.8)
X_train = Tfidf_vectorizer.fit_transform(utils.back_to_doc(text_train)[:train_len])
Y_train = label_total[:train_len]
X_test = Tfidf_vectorizer.transform(utils.back_to_doc(text_test))
Y_test = label_total[train_len:]
print(np.shape(X_train))

In [13]:
SVC_clf = load('py37_SVC_clf_12gram.joblib')
y_pred = SVC_clf.predict(X_test)
y_pred_proba = SVC_clf.predict_proba(X_test)
y_pred = np.argmax(y_pred_proba,axis=-1)
print(metrics.classification_report(Y_test, y_pred, target_names=['No_infection','infection']))
print(metrics.confusion_matrix(Y_test,y_pred))
print(metrics.roc_auc_score(Y_test, y_pred))

FileNotFoundError: [Errno 2] No such file or directory: 'py37_SVC_clf_12gram.joblib'

In [10]:
type(y_pred)

numpy.ndarray

In [12]:
np.save('svm_y_pred.npy',y_pred)

In [19]:
Y_train = label_total[:train_len]
print(np.shape(Y_train))

(2887,)


In [10]:
SVC_clf = svm.SVC(class_weight = 'balanced',kernel = 'linear',probability=1)
SVC_clf.fit(X_train, Y_train)
y_pred = SVC_clf.predict(X_test)
y_pred_proba = SVC_clf.predict_proba(X_test)
y_pred = np.argmax(y_pred_proba,axis=-1)
print(metrics.classification_report(Y_test, y_pred, target_names=['No_PU','PU']))
print(metrics.confusion_matrix(Y_test,y_pred))
print(metrics.roc_auc_score(Y_test, y_pred))

              precision    recall  f1-score   support

No_infection       0.87      0.91      0.89       711
   infection       0.79      0.73      0.76       356

    accuracy                           0.85      1067
   macro avg       0.83      0.82      0.82      1067
weighted avg       0.84      0.85      0.84      1067

[[644  67]
 [ 97 259]]
0.8166473079536654


In [ ]:
SVC_clf = svm.SVC(class_weight = 'balanced',kernel = 'linear',probability=1)
SVC_clf.fit(X_train, Y_train)
y_pred = SVC_clf.predict(X_test)
y_pred_proba = SVC_clf.predict_proba(X_test)
y_pred = np.argmax(y_pred_proba,axis=-1)
print(metrics.classification_report(Y_test, y_pred, target_names=['No_PU','PU']))
print(metrics.confusion_matrix(Y_test,y_pred))
print(metrics.roc_auc_score(Y_test, y_pred))

In [12]:
X_test.shape

(722, 83475)

In [23]:
dump(SVC_clf, 'SVC_clf_12gram.joblib')

['SVC_clf_12gram.joblib']

In [24]:
LR = LogisticRegression(max_iter=1000)
LR.fit(X_train,Y_train)
y_pred = LR.predict(X_test)

y_pred_proba = LR.predict_proba(X_test)
y_pred = np.argmax(y_pred_proba,axis=-1)

print(metrics.classification_report(Y_test, y_pred, target_names=['No_infection','infection']))
print(metrics.confusion_matrix(Y_test,y_pred))
print(metrics.roc_auc_score(Y_test, y_pred))

              precision    recall  f1-score   support

No_infection       0.85      0.96      0.90       481
   infection       0.90      0.65      0.76       241

    accuracy                           0.86       722
   macro avg       0.87      0.81      0.83       722
weighted avg       0.87      0.86      0.85       722

[[464  17]
 [ 84 157]]
0.8080546234073206


In [25]:
dump(LR, 'LR_clf_12gram.joblib')

['LR_clf_12gram.joblib']

# 1 gram SVM

In [67]:
Tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,1),min_df =5,stop_words='english',max_df = 0.8)
X_train = Tfidf_vectorizer.fit_transform(new_text_train)
Y_train = label_total[:train_len]
X_test = Tfidf_vectorizer.transform(new_text_test)
Y_test = label_total[train_len:]
print(np.shape(X_train))

(2887, 11741)


In [68]:
np.shape(Y_train)

(2887,)

In [69]:
SVC_clf_1gram = svm.SVC(class_weight = 'balanced',kernel = 'linear',probability=1)
SVC_clf_1gram.fit(X_train,Y_train)
y_pred = SVC_clf_1gram.predict(X_test)

y_pred_proba = SVC_clf_1gram.predict_proba(X_test)
y_pred = np.argmax(y_pred_proba,axis=-1)

print(metrics.classification_report(Y_test, y_pred, target_names=['No_infection','infection']))
print(metrics.confusion_matrix(Y_test,y_pred))
print(metrics.roc_auc_score(Y_test, y_pred))

              precision    recall  f1-score   support

No_infection       0.88      0.93      0.91       481
   infection       0.85      0.76      0.80       241

    accuracy                           0.87       722
   macro avg       0.87      0.84      0.85       722
weighted avg       0.87      0.87      0.87       722

[[448  33]
 [ 59 182]]
0.8432898266923163


In [32]:
dump(SVC_clf_1gram, 'SVC_clf_1gram.joblib')

['SVC_clf_1gram.joblib']

In [30]:
LR_1gram = LogisticRegression(max_iter=1000)
LR_1gram.fit(X_train,Y_train)
y_pred = LR_1gram.predict(X_test)

y_pred_proba = LR_1gram.predict_proba(X_test)
y_pred = np.argmax(y_pred_proba,axis=-1)

print(metrics.classification_report(Y_test, y_pred, target_names=['No_infection','infection']))
print(metrics.confusion_matrix(Y_test,y_pred))
print(metrics.roc_auc_score(Y_test, y_pred))

              precision    recall  f1-score   support

No_infection       0.86      0.96      0.91       481
   infection       0.91      0.69      0.78       241

    accuracy                           0.87       722
   macro avg       0.88      0.83      0.85       722
weighted avg       0.88      0.87      0.87       722

[[464  17]
 [ 75 166]]
0.826726822577445


In [33]:
dump(LR_1gram, 'LR_clf_1gram.joblib')

['LR_clf_1gram.joblib']

In [70]:
import imp
imp.reload(utils)
pos_words = utils.show_word_new(Tfidf_vectorizer,SVC_clf_1gram,100,for_pos = True)
print(pos_words)

0.8539091962955342
['wound', 'cellulitis', 'sternal', 'erythema', 'abscess', 'vancomycin', 'site', 'open', 'collection', 'knee', 'post', 'laparotomy', 'infection', 'antibiotic', 'exploratory', 'incision', 'surgery', 'id', 'trough', 'washout', 'culture', 'keflex', 'hardware', 'bone', 'pressor', 'dress', '14', 'leak', 'ercp', 'rehab', 'whipple', 'hip', 'duodenum', 'linezolid', 'complicate', 'undergo', 'biopsy', 'cholecystectomy', 'fluid', 'vegetation', 'develop', 'continue', 'start', 'osteomyelitis', 'orthopedic', 'debridement', 'dehiscence', 'grow', 'tibia', 'ileostomy', 'colectomy', 'flap', 'drain', 'suppository', 'repair', 'colic', 'line', 'cbd', 'elevate', 'placement', 'inr', 'augmentin', 'pod', 'perforation', 'resection', 'femoral', 'worsen', '12h', '500', '70', 'infectious', 'charcot', 'place', 'prn', 'growth', 'vac', 'anastomosis', 'nafcillin', 'ankle', '53am', 'hemi', 'soln', 'gram', '78', 'midodrine', 'bilateral', 'support', 'goal', 'unasyn', 'cephalexin', 'slowly', 'laparoscopi

In [71]:
neg_words = utils.show_word_new(Tfidf_vectorizer,SVC_clf_1gram,100,for_pos = False)
print(neg_words)

0.6439329484590215
['ulcer', 'pancreatitis', 'intact', 'urinary', 'unremarkable', '50', 'initially', 'sided', 'pressure', 'hct', 'baseline', 'hcl', 'hemodialysis', 'negative', 'carcinoma', 'viral', 'avm', 'insufficiency', 'esrd', 'known', 'peg', 'intervention', '40am', 'denie', '20pm', 'obesity', 'father', 'bp', 'mediastinal', 'stool', 'igg', 'mvr', 'ed', 'weakness', 'regimen', 'contrast', 'vessel', 'coronary', 'pericardial', 'severely', 'arterial', 'cefepime', 'center', 'cataract', 'sob', 'rapid', 'paraesophageal', '91', 'pa', '31pm', 'ranitidine', 'dialysis', 'carotid', 'sinus', 'retrocardiac', 'thoracotomy', 'monitoring', 'c6', 'myxoma', 'diverticulitis', 'skull', 'pseudocyst', 'peak', 'cough', 'neuro', 'occlusion', 'nt', 'nasogastric', 'sputum', 'foley', 'release', 'venous', 'trend', 'dysfunction', 'fio2', '48pm', 'glaucoma', 'husband', 'independently', 'concern', 'consistent', 'lactate', 'cocaine', 'mri', 'dilate', 'answer', '68', 'ambulatory', 'minimally', 'breathe', 'tab', 'bila

In [11]:
import imp
imp.reload(utils)
pos_words = utils.show_word_new(Tfidf_vectorizer,LR_1gram,100,for_pos = True)
print(pos_words)

1.2316297894203434
['uti', 'tract', 'urinary', 'sepsis', 'infection', 'ciprofloxacin', 'diff', 'difficile', 'septic', 'positive', 'cipro', 'treat', 'urine', 'ua', 'klebsiella', 'pseudomonas', 'start', 'culture', 'grow', 'clarithromycin', '500', 'nitrofurantoin', 'nafcillin', 'colitis', 'mod', 'urinalysis', 'vancomycin', 'amoxicillin', 'shock', 'flagyl', 'pneumonia', 'staph', 'pylori', 'expire', 'pressor', 'vanco', 'bactrim', 'levofloxacin', 'organisms', 'bacteria', 'clostridium', '000', 'hypotension', 'metronidazole', 'enterococcus', 'failure', 'sensitive', 'complete', 'ml', 'coagulase', 'levophe', 'wbc', 'ostomy', 'coli', 'place', 'decortication', 'yeast', 'cefpodoxime', 'develop', 'vasopressor', 'eventually', 'daughter', 'sputum', 'q12h', 'placement', 'tracheostomy', 'fall', 'cdiff', 'urosepsis', 'pylorus', 'central', 'hypotensive', 'mssa', 'require', 'zosyn', 'female', 'pneumoniae', 'wheelchair', 'expired', 'spectrum', 'pan', 'mrs', 'picc', 'multiple', 'resuscitate', 'resuscitation'

In [82]:
lines = open('/home/zilong.zhang1/AE_automation/BERT/pretrained_bert_tf/biobert_pretrain_output_all_notes_150000/vocab.txt','r',encoding="utf-8").readlines()

In [86]:
vocab_dict = {}
for line in lines:
    token = line.strip()
    vocab_dict[token] =1


In [87]:
for www in pos_words:
    if www not in vocab_dict:
        print(www)

cellulitis
sternal
erythema
abscess
vancomycin
laparotomy
antibiotic
exploratory
incision
trough
washout
keflex
pressor
ercp
rehab
whipple
duodenum
linezolid
complicate
biopsy
cholecystectomy
osteomyelitis
orthopedic
debridement
dehiscence
tibia
ileostomy
colectomy
suppository
colic
cbd
elevate
inr
augmentin
perforation
resection
femoral
worsen
12h
charcot
prn
vac
anastomosis
nafcillin
53am
hemi
soln
midodrine
unasyn
cephalexin
laparoscopic
sepsis
cmo
ngt


In [89]:
for www in neg_words:
    if www not in vocab_dict:
        print(www)

ulcer
pancreatitis
urinary
unremarkable
hct
baseline
hcl
hemodialysis
carcinoma
avm
insufficiency
esrd
peg
40am
denie
20pm
obesity
bp
mediastinal
igg
mvr
regimen
coronary
pericardial
arterial
cefepime
cataract
paraesophageal
pa
31pm
ranitidine
dialysis
carotid
sinus
retrocardiac
thoracotomy
c6
myxoma
diverticulitis
pseudocyst
neuro
occlusion
nt
nasogastric
sputum
foley
venous
dysfunction
fio2
48pm
glaucoma
lactate
mri
dilate
ambulatory
minimally
tab
bilat
12pm
codeine
extraocular
stenting


In [34]:
import sklearn
sklearn

<module 'sklearn' from '/home/zilong.zhang1/miniconda3/envs/py35/lib/python3.5/site-packages/sklearn/__init__.py'>

In [37]:
# Grid Search for NB parameters
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline_helper import PipelineHelper
X = new_text_total
Y = label_total
pipe = Pipeline([
    ('vectorizer', PipelineHelper([
        ('count', CountVectorizer(stop_words='english')),
        ('tfidf', TfidfVectorizer(stop_words='english')),
    ])),
    ('classifier', PipelineHelper([
        ('Multinomial', MultinomialNB()),
        ('Complement', ComplementNB()),
    ])),
])

params = {
    'vectorizer__selected_model': pipe.named_steps['vectorizer'].generate({
        'count__ngram_range': [(1,1),(1,2),(1,3)],
        'count__min_df': [5,10],
        'count__max_df': [0.4,0.6,0.8],
        'tfidf__ngram_range': [(1,1),(1,2),(1,3)],
        'tfidf__min_df':[5,10],
        'tfidf__max_df':[0.4,0.6,0.8],
    }),
    'classifier__selected_model': pipe.named_steps['classifier'].generate({
        'Multinomial__alpha': [1.0],
        'Complement__alpha': [1.0],
    })
}
grid = GridSearchCV(pipe, params, scoring='accuracy', verbose=1,cv=5,n_jobs=-1)
grid.fit(X, Y)
print(grid.best_params_)
print(grid.best_score_)


Fitting 5 folds for each of 72 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  1.2min
exception calling callback for <Future at 0x7f68cb7c0a20 state=finished raised TerminatedWorkerError>
Traceback (most recent call last):
  File "/home/zilong.zhang1/miniconda3/envs/py35/lib/python3.5/site-packages/joblib/externals/loky/_base.py", line 625, in _invoke_callbacks
    callback(self)
  File "/home/zilong.zhang1/miniconda3/envs/py35/lib/python3.5/site-packages/joblib/parallel.py", line 340, in __call__
    self.parallel.dispatch_next()
  File "/home/zilong.zhang1/miniconda3/envs/py35/lib/python3.5/site-packages/joblib/parallel.py", line 769, in dispatch_next
    if not self.dispatch_one_batch(self._original_iterator):
  File "/home/zilong.zhang1/miniconda3/envs/py35/lib/python3.5/site-packages/joblib/parallel.py", line 835, in dispatch_one_batch
    self._dispatch(tasks)
  File "/home/zilong.zhang1/miniconda3/envs/py35/lib/python3.5/site-

TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker. The exit codes of the workers are {SIGKILL(-9)}

In [ ]:
print(grid.best_params_)

In [ ]:
Tfidf_vectorizer =TfidfVectorizer(ngram_range=(1,1),min_df =10,stop_words='english',token_pattern=r"(?u)\b\d?[a-z]{2,}\d?\b",max_df = 0.8)
Tfidf_vec = Tfidf_vectorizer.fit_transform(new_text_total)
print(np.shape(Tfidf_vec))
X_train, X_test, y_train, y_test = train_test_split(Tfidf_vec, label_total, test_size=0.2, random_state=41)
clf = MultinomialNB()
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print(metrics.classification_report(y_test, y_pred, target_names=['No_pressure_ulcer','ressure_ulcer']))
print(metrics.confusion_matrix(y_test,y_pred))

In [ ]:
from sklearn.metrics import roc_curve, auc
import sklearn
y_pred_prob2 = y_pred_proba[:,1]
fpr,tpr,thres = roc_curve(y_test,y_pred_prob2)
print(sklearn.metrics.roc_auc_score(y_test, y_pred_prob2))

import matplotlib.pyplot as plt
plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr, tpr, label='SVM')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()

In [ ]:
def _svm_pipeline_1gram(global_idf):
    Tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,1),min_df =5,stop_words='english',max_df = 0.8)
    #--------------------Vectorize----------------------
    # Use the idf of whole text or only the idf of training dataset
    if global_idf:
        Tfidf_vectorizer = Tfidf_vectorizer.fit(new_text_total)
    else:
        Tfidf_vectorizer = Tfidf_vectorizer.fit(new_text_train)
    X_train = Tfidf_vectorizer.transform(new_text_train)
    y_train = label_train
    X_test = Tfidf_vectorizer.transform(new_text_test)
    y_test = label_test
    print(np.shape(X_train))
    #--------------------Training----------------------
    SVC_clf = svm.SVC(class_weight = 'balanced',kernel = 'linear',probability=1)
    SVC_clf.fit(X_train,y_train)
    #--------------------Evaluation----------------------
    y_pred = SVC_clf.predict(X_test)

    y_pred_proba = SVC_clf.predict_proba(X_test)
    y_pred = np.argmax(y_pred_proba,axis=-1)

    print(metrics.classification_report(y_test, y_pred, target_names=['No_pressure_ulcer','pressure_ulcer']))
    print(metrics.confusion_matrix(y_test,y_pred))
    
    y_pred_prob2 = y_pred_proba[:,1]
    fpr,tpr,thres = roc_curve(y_test,y_pred_prob2)
    print(sklearn.metrics.roc_auc_score(y_test, y_pred_prob2))
    return sklearn.metrics.roc_auc_score(y_test, y_pred_prob2)

In [4]:
training_generator,validation_generator,dummy_generator = utils.load_data_new()
for aaa in validation_generator:
    pass